Импортируем всякое

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

Пока не поправили сам файл никакие энжены и пропуски битых строк не помогли!

In [2]:
data=pd.read_csv("twits.csv")
data

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


Решил переименовать столбец с текстом твита, ну и столбец с меткой тоже

In [3]:
data.rename(columns={data.columns[0]:'label',data.columns[5]:'text'},inplace=True)
data.head()

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [4]:
data['label'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1599999 entries, 0 to 1599998
Series name: label
Non-Null Count    Dtype
--------------    -----
1599999 non-null  int64
dtypes: int64(1)
memory usage: 12.2 MB


Я думал что при обучении в метку запишется то значение которое мы укажем, т.е. указали, что позитивный твит - 100 - значит и будет 100, но потом там ниже начались проблемы побрались откуда-то 1, поэтому присвою ка я всем позитивным 1, а негативным 0!

In [5]:
data['label'].value_counts()

label
4    800000
0    799999
Name: count, dtype: int64

In [6]:
data.loc[data['label']==4,'label']=1
data['label'].value_counts()

label
1    800000
0    799999
Name: count, dtype: int64

In [7]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


По совету друзей для повышения однородности текста очистим его с помощью таких нехитрых ф-ий. (P.S. - эффекта это никакого не дало!)

In [8]:
import re
import emoji

def cleareText(text):
    text=re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text=re.sub(r"[^A-Za-z0-9\s]", '', text)
    text=emoji.replace_emoji(text, replace='')
    return text.lower()

data['text']=data['text'].apply(cleareText)
data.sample(10)

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text
961963,1,1827197273,Sun May 17 10:11:56 PDT 2009,NO_QUERY,chalinfernando,just got back from hyde park hoping for good ...
880991,1,1685727811,Sun May 03 01:46:24 PDT 2009,NO_QUERY,CherylH77,mrflossy ive learnt my lesson about driving in...
915548,1,1753135278,Sun May 10 00:16:25 PDT 2009,NO_QUERY,kimlyclarete,im feeling great
410366,0,2059858134,Sat Jun 06 17:46:42 PDT 2009,NO_QUERY,rxgellivictor,rxgcrew sino in charge sa tweets dito nangaasa...
839525,1,1559451338,Sun Apr 19 11:10:31 PDT 2009,NO_QUERY,DarkPiano,pianogeek hey back atcha
224526,0,1977436541,Sat May 30 20:58:37 PDT 2009,NO_QUERY,LisaMMH,too bad lebron lakers all the way lisamm
801528,1,1468087930,Mon Apr 06 23:38:48 PDT 2009,NO_QUERY,_DanielTouch_,is eating frozen grapes in an effort to fill i...
1243037,1,1994455399,Mon Jun 01 11:57:00 PDT 2009,NO_QUERY,CrystalMOwens,queensapphyre ooo sounds like fun i hope you h...
241636,0,1981132112,Sun May 31 08:29:28 PDT 2009,NO_QUERY,bowiesarea,no more th hampm faa shirts
1277781,1,2001360969,Tue Jun 02 00:38:02 PDT 2009,NO_QUERY,loveandbooze,getting up in 5 hours


Все как и было на семинаре - делим 20 на 80

In [9]:
labeled_twits, unlabeled_twits = train_test_split(data,train_size=0.2, random_state=31)
labeled_twits

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text
252084,0,1983639045,Sun May 31 13:32:38 PDT 2009,NO_QUERY,fitzbeew,is frustrated and feeling helpless ochem time
125100,0,1834246879,Mon May 18 02:11:43 PDT 2009,NO_QUERY,daniec22,is in work
518207,0,2191483305,Tue Jun 16 05:22:13 PDT 2009,NO_QUERY,ccjones2005,i learned last night that lifting weights run...
155848,0,1955963000,Thu May 28 20:56:03 PDT 2009,NO_QUERY,sweetface35,aplusk ashton why do you not respond to me
973910,1,1833310348,Sun May 17 22:53:51 PDT 2009,NO_QUERY,annalisaceccato,got soccer training this arvo and soccer game ...
...,...,...,...,...,...,...
462487,0,2174707331,Sun Jun 14 23:07:57 PDT 2009,NO_QUERY,garrypettet,has got a sore throat
1260130,1,1998268607,Mon Jun 01 18:16:15 PDT 2009,NO_QUERY,jessprewitt,dmb live now fuse or hulu new cd drops tomorrow
289662,0,1995081307,Mon Jun 01 12:55:14 PDT 2009,NO_QUERY,burjz,biggorillavoice oh crap christian bale would b...
419856,0,2062078475,Sat Jun 06 22:16:59 PDT 2009,NO_QUERY,XxmileyjonasxX,my bestie and i had a fight over something soo...


Создаем этот векторайзер и модель для обучения на основе размеченых данных

In [10]:
def train_model(labeled_reviews):
    # Векторизация текстовых данных с помощью TF-IDF
    vectorizer = TfidfVectorizer()
    x = vectorizer.fit_transform(labeled_reviews['text'])
    y = labeled_reviews['label']
    # Обучение модели логистической регрессии на размеченных данных
    model = LogisticRegression()
    model.fit(x, y)
    return model, vectorizer

model, vectorizer=train_model(labeled_twits)

Векторизуем и размечаем "неразмеченные"

In [11]:
x_unlabeled = vectorizer.transform(unlabeled_twits['text'])
y_unlabeled_predicted = model.predict(x_unlabeled)

Получаем меру неуверенности модели в своем выборе

In [12]:
y_unlabeled_proba = model.predict_proba(x_unlabeled)
uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)

uncertainty.argsort() - сортирует по-возрастанию,срезом [:10000] - получаем от 0 до 10000.  Поскольку данных много - взяли 10000 самых неуверенных ответов модели, т.е. значений с самой "низкой уверенностью"

In [13]:
labeled_reviews_new = unlabeled_twits.iloc[uncertainty.argsort()[:10000]]
unlabeled_reviews_new = unlabeled_twits.iloc[uncertainty.argsort()[10000:]]

In [14]:
labeled_reviews_new 


,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text
735183,0,2264723697,Sun Jun 21 04:45:25 PDT 2009,NO_QUERY,onNsmind,is sad i miss u
342088,0,2015185160,Wed Jun 03 03:57:27 PDT 2009,NO_QUERY,shitHappens1989,sad and i miss u
504342,0,2188102378,Mon Jun 15 21:18:22 PDT 2009,NO_QUERY,caseymariex,im sad i miss my mittenlove24
330079,0,2011837196,Tue Jun 02 19:38:46 PDT 2009,NO_QUERY,alexandralsims,holliehixson sad i miss this
461258,0,2174404509,Sun Jun 14 22:29:02 PDT 2009,NO_QUERY,tjane1995,all change mahmind i dont want the school ends...
...,...,...,...,...,...,...
799987,0,2329204559,Thu Jun 25 10:28:26 PDT 2009,NO_QUERY,court_lyn,farrah fawcett passed awayso sad
334597,0,2013612113,Tue Jun 02 23:06:32 PDT 2009,NO_QUERY,thefluentone,andeepandee i tried sorry
377831,0,2051976427,Fri Jun 05 23:18:17 PDT 2009,NO_QUERY,michxxblc,luvinmesomed totally sad eeee lets not think ...
380641,0,2052611606,Sat Jun 06 01:20:39 PDT 2009,NO_QUERY,Fugghedabouddit,terrible tummy ache not hungry not diarrhea no...


Обьединяем их с нашим размеченым пулом. Как-будьто мы просмотрели эти "неуверенные" данные и вручную разметили

In [15]:
labeled_twits = pd.concat([labeled_twits, labeled_reviews_new])

In [16]:
print(labeled_twits.shape[0])

329999


контролируем сколько у нас данных считаются размеченными

Снова обучаем модель!

In [17]:
model, vectorizer = train_model(labeled_twits)

И наконец-то применяем модель к данным. Получаем предсказание модели - запишем его в столбик 'sentinel'. Также считаем чет типа вероятности попадания - f1 

In [18]:
x_test = vectorizer.transform(data['text'])
y_test_predicted = model.predict(x_test)
data['sentinel']=y_test_predicted
f1=f1_score(data['label'], y_test_predicted)
data

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text,sentinel
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...,0
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan i dived many times for the ball manag...,1
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass no its not behaving at all im ...,0
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,kwesidei not the whole crew,0
...,...,...,...,...,...,...,...
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,just woke up having no school is the best feel...,0
1599995,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,thewdbcom very cool to hear old walt intervie...,1
1599996,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,are you ready for your mojo makeover ask me fo...,1
1599997,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,happy 38th birthday to my boo of alll time tup...,0


In [19]:
f1

0.8033524493730902

Выше этот f1, а ниже считаем дедовским способом - берем те строки где label и sentinel не равны и получаем в процентах 

In [20]:
temp=data[data['sentinel']!=data['label']]
temp

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text,sentinel
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan i dived many times for the ball manag...,1
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,loltrish hey long time no see yes rains a bit...,1
10,0,1467812579,Mon Apr 06 22:20:17 PDT 2009,NO_QUERY,pardonlauren,i just repierced my ears,1
17,0,1467813579,Mon Apr 06 22:20:31 PDT 2009,NO_QUERY,starkissed,lettya ahh ive always wanted to see rent love...,1
18,0,1467813782,Mon Apr 06 22:20:34 PDT 2009,NO_QUERY,gi_gi_bee,fakerpattypattz oh dear were you drinking out ...,1
...,...,...,...,...,...,...,...
1599958,1,2193577680,Tue Jun 16 08:38:51 PDT 2009,NO_QUERY,WParenthetical,ashinynewcoin yeah thatd be the one sorry to ...,0
1599972,1,2193578319,Tue Jun 16 08:38:55 PDT 2009,NO_QUERY,luckygeorgeblog,omg office space i wanna steal it,0
1599990,1,2193579284,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,AgustinaP,rmedina latati mmmm that sounds absolutely pe...,0
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,just woke up having no school is the best feel...,0


Ну, или где равны. Почти равно f1!

In [21]:
r=100-(temp.shape[0]*100)/data.shape[0]
r

80.1824251140157

Маловато будет! Давайте попробуем дообучить нашу модель!

In [22]:
#Проверка
def check():
    global vectorizer,model
    x_test = vectorizer.transform(data['text'])
    y_test_predicted = model.predict(x_test)
    f1=f1_score(data['label'], y_test_predicted)
    print(f1)
    return f1
    
#Тренеровка
def training():
    global vectorizer,model,unlabeled_twits,labeled_twits
    x_unlabeled = vectorizer.transform(unlabeled_twits['text'])
    y_unlabeled_predicted = model.predict(x_unlabeled)
    
    y_unlabeled_proba = model.predict_proba(x_unlabeled)
    uncertainty = -(y_unlabeled_proba * np.log2(y_unlabeled_proba)).sum(axis=1)
    
    labeled_reviews_new = unlabeled_twits.iloc[uncertainty.argsort()[:100000]]
    # unlabeled_twits = unlabeled_twits.iloc[uncertainty.argsort()[100000:]]
    
    labeled_twits = pd.concat([labeled_twits, labeled_reviews_new])
    print(labeled_twits.shape[0])
    model, vectorizer = train_model(labeled_twits)
    
n=0
while check()<0.9 and n<5:
    training()
    n+=1

print("Done!")


0.8033524493730902
429999
0.8027625816002271
529999
0.798918671542472
629999
0.7987508584768235
729999
0.7982857910770889
829999
0.7986930014183191
Done!


Во дела! Модель то не обучается, а разучается! Ну, мы хотябы попытались!

In [23]:
x_test = vectorizer.transform(data['text'])
y_test_predicted = model.predict(x_test)
data['sentinel']=y_test_predicted
f1=f1_score(data['label'], y_test_predicted)
temp=data[data['sentinel']!=data['label']]
r=100-(temp.shape[0]*100)/data.shape[0]
data

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text,sentinel
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he cant update his facebook by t...,0
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan i dived many times for the ball manag...,1
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,nationwideclass no its not behaving at all im ...,0
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,kwesidei not the whole crew,0
...,...,...,...,...,...,...,...
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,just woke up having no school is the best feel...,0
1599995,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,thewdbcom very cool to hear old walt intervie...,1
1599996,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,are you ready for your mojo makeover ask me fo...,1
1599997,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,happy 38th birthday to my boo of alll time tup...,0


In [24]:
temp

,label,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,text,sentinel
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,kenichan i dived many times for the ball manag...,1
6,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,loltrish hey long time no see yes rains a bit...,1
10,0,1467812579,Mon Apr 06 22:20:17 PDT 2009,NO_QUERY,pardonlauren,i just repierced my ears,1
17,0,1467813579,Mon Apr 06 22:20:31 PDT 2009,NO_QUERY,starkissed,lettya ahh ive always wanted to see rent love...,1
18,0,1467813782,Mon Apr 06 22:20:34 PDT 2009,NO_QUERY,gi_gi_bee,fakerpattypattz oh dear were you drinking out ...,1
...,...,...,...,...,...,...,...
1599972,1,2193578319,Tue Jun 16 08:38:55 PDT 2009,NO_QUERY,luckygeorgeblog,omg office space i wanna steal it,0
1599983,1,2193578847,Tue Jun 16 08:38:57 PDT 2009,NO_QUERY,RobFoxKerr,you heard it here first were having a girl ho...,0
1599990,1,2193579284,Tue Jun 16 08:38:59 PDT 2009,NO_QUERY,AgustinaP,rmedina latati mmmm that sounds absolutely pe...,0
1599994,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,just woke up having no school is the best feel...,0


In [25]:
f1

0.7986930014183191

In [26]:
r

79.58817474260921